# Rayleigh-Bénard Convection in an annulus



In [1]:
import petsc4py
from petsc4py import PETSc

import underworld3 as uw
from underworld3.systems import Stokes
from underworld3 import function

import numpy as np
import sympy
import os

# to fix trame issue
import nest_asyncio
nest_asyncio.apply()

[Lyrebird.local:26748] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.Lyrebird.501/jf.0/606339072/sm_segment.Lyrebird.501.24240000.0 could be created.


In [2]:
# The problem setup 

r_o = 1
r_i = 0.5


# Parameters that define the notebook
# These can be set when launching the script as
# mpirun python3 scriptname -uw_resolution=0.1 etc

rayleigh_number_exponent = uw.options.getReal("rayleigh_no_expt",6) 
resolution = uw.options.getInt("model_resolution", default=25)
max_steps = uw.options.getInt("max_steps", default=50)
restart_step = uw.options.getInt("restart_step", default=-1)

# How that works

rayleigh_number = pow(10,rayleigh_number_exponent)
expt_name = f"Ra1e{rayleigh_number_exponent}_res{resolution}"
output_dir = os.path.join("output","annulus",f"Ra1e{rayleigh_number_exponent}")

os.makedirs(output_dir, exist_ok=True)


In [3]:
## Set up the mesh geometry / discretisation


meshball = uw.meshing.Annulus(
    radiusInner=r_i, 
    radiusOuter=r_o, 
    cellSize=1/resolution, 
    degree=1, 
    qdegree=4, 
)

x,y  = meshball.CoordinateSystem.X
r, th = meshball.CoordinateSystem.R
r_vector = meshball.CoordinateSystem.unit_e_0


In [4]:
v_soln = uw.discretisation.MeshVariable("U", meshball, meshball.dim, degree=2)
p_soln = uw.discretisation.MeshVariable("P", meshball, 1, degree=1)
t_soln = uw.discretisation.MeshVariable("T", meshball, 1, degree=3)

In [5]:
# passive_swarm = uw.swarm.Swarm(mesh=meshball)
# passive_swarm.populate(fill_param=3)

In [6]:
# Create solver to solver the momentum equation (Stokes flow)

stokes = Stokes(
    meshball, 
    velocityField=v_soln, 
    pressureField=p_soln, 
    solver_name="stokes",
)

stokes.tolerance = 0.001

stokes.constitutive_model = uw.constitutive_models.ViscousFlowModel
stokes.constitutive_model.Parameters.viscosity = 1.0

# Penalise flow crossing the boundary

stokes.add_natural_bc(10*rayleigh_number * v_soln.sym.dot(r_vector) * r_vector, "Upper")
stokes.add_natural_bc(10*rayleigh_number * v_soln.sym.dot(r_vector) * r_vector, "Lower")

stokes.bodyforce = r_vector * rayleigh_number * t_soln.sym[0] / (0.5) ** 3

In [7]:
# Create solver for the energy equation (Advection-Diffusion of temperature)

adv_diff = uw.systems.AdvDiffusion(
    meshball,
    u_Field=t_soln,
    V_fn=v_soln,
    solver_name="adv_diff",
    order=2,
)

adv_diff.constitutive_model = uw.constitutive_models.DiffusionModel
adv_diff.constitutive_model.Parameters.diffusivity = 1

## Boundary conditions for this solver

adv_diff.add_dirichlet_bc(1.0, "Lower")
adv_diff.add_dirichlet_bc(0.0, "Upper")

In [8]:
# The advection / diffusion equation is an initial value problem
# We set this up to have 

abs_r = sympy.sqrt(meshball.rvec.dot(meshball.rvec))
init_t = 0.05 * sympy.sin(2.0 * th) * sympy.sin(np.pi * (r - r_i) / (r_o - r_i)) + (
    r_o - r
) / (r_o - r_i)


with meshball.access(t_soln):
    t_soln.data[...] = uw.function.evaluate(init_t, t_soln.coords).reshape(-1, 1)


In [9]:
# Solution strategy: solve for the velocity field, then update the temperature field.
# First we check this works for a tiny timestep, later we will repeat this to 
# model the passage of time.

stokes.solve()
adv_diff.solve(timestep=0.00001 * stokes.estimate_dt())

In [12]:
0/0

ZeroDivisionError: division by zero

In [ ]:
timestep = 0
elapsed_time=0.0

In [ ]:
# Convection model / update in time

output = os.path.join(output_dir, expt_name)

for step in range(0, max_steps):
    stokes.solve()
    delta_t = 2.0 * stokes.estimate_dt()
    adv_diff.solve(timestep=delta_t)

    # stats then loop
    tstats = t_soln.stats()

    if uw.mpi.rank == 0:
        print(f"Timestep {timestep}, dt {delta_t}, t {elapsed_time}")

    meshball.write_timestep(filename="annulus_ckpt",
                            index=timestep,
                            outputPath=output_dir,
                            meshVars=[v_soln, p_soln, t_soln],
                        )

    timestep += 1
    elapsed_time += delta_t


In [ ]:
if uw.mpi.size == 1:
    
    import pyvista as pv
    import underworld3.visualisation as vis

    pvmesh = vis.mesh_to_pv_mesh(meshball)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym)
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    velocity_points = vis.meshVariable_to_pv_cloud(stokes.u)
    velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, stokes.u.sym)

    # point sources at cell centres
    cpoints = pvmesh.cell_centers().points
    cpoint_cloud = pv.PolyData(cpoints)

    pvstream = pvmesh.streamlines_from_source(
            cpoint_cloud,
            vectors="V",
            integrator_type=2,
            integration_direction="both",
            compute_vorticity=False,
            max_steps=1000,
            max_time=0.1,
            initial_step_length=0.01,
            surface_streamlines=True,
            )

    pl = pv.Plotter(window_size=(750, 750))

    pl.add_mesh(pvstream, opacity=0.5)
    pl.add_mesh(pvmesh, scalars="T", cmap="RdBu_r", opacity=1)

    pl.show(cpos="xy")

In [ ]:
0/0

In [ ]:
adv_diff.DuDt._psi_star_projection_solver.u